In [1]:
import pandas as pd
import numpy as np

In [2]:
credit = pd.read_excel('credit_data.xlsx')

In [3]:
credit.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [4]:
credit['Job_des'] = credit['Job'].map({0:'unskilled_no_resident',1:'unskilled_resident',2:'skilled',3:'highly_skilled'})

In [5]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               1000 non-null   int64 
 3   Housing           1000 non-null   object
 4   Saving accounts   817 non-null    object
 5   Checking account  606 non-null    object
 6   Credit amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
 10  Job_des           1000 non-null   object
dtypes: int64(4), object(7)
memory usage: 86.1+ KB


In [6]:
credit_woe = credit.drop('Job',axis=1)

In [7]:
credit_woe.pivot_table(index='Risk',columns='Saving accounts', values='Job_des',aggfunc='count')

Saving accounts,little,moderate,quite rich,rich
Risk,,,,
bad,217,34,11,6
good,386,69,52,42


In [8]:
credit_woe.pivot_table(index='Risk',columns='Checking account', values='Job_des',aggfunc='count')

Checking account,little,moderate,rich
Risk,,,
bad,135,105,14
good,139,164,49


In [9]:
credit_woe['Checking account'] = np.where(credit_woe['Checking account'].isnull(),
                                          np.where(credit['Risk']=='bad','little','moderate'),
                                          credit_woe['Checking account'])

In [10]:
credit_woe['Saving accounts'] = np.where(credit_woe['Saving accounts'].isnull(),
                                          'little',
                                          credit_woe['Saving accounts'])

In [11]:
credit_woe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Housing           1000 non-null   object
 3   Saving accounts   1000 non-null   object
 4   Checking account  1000 non-null   object
 5   Credit amount     1000 non-null   int64 
 6   Duration          1000 non-null   int64 
 7   Purpose           1000 non-null   object
 8   Risk              1000 non-null   object
 9   Job_des           1000 non-null   object
dtypes: int64(3), object(7)
memory usage: 78.2+ KB


In [12]:
credit_woe.describe()

,Age,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000
mean,35.546000,3271.258000,20.903000
std,11.375469,2822.736876,12.058814
min,19.000000,250.000000,4.000000
25%,27.000000,1365.500000,12.000000
50%,33.000000,2319.500000,18.000000
75%,42.000000,3972.250000,24.000000
max,75.000000,18424.000000,72.000000


In [13]:
def age_eval(x):
    if x <= 27:
        return '< 27'
    elif x <= 33:
        return '27 - 33'
    elif x <= 42:
        return '33 - 42'
    elif x <= 52:
        return '42 - 52'
    else:
        return '>52'

In [14]:
def amt_eval(x):
    if x <= 1000:
        return '< 1,000'
    elif x <= 2000:
        return '1,000 - 2,000'
    elif x <= 3000:
        return '2,000 - 3,000'
    elif x <= 4000:
        return '3,000 - 4,000'
    elif x <= 6000:
        return '4,000 - 6,000'
    elif x <= 10000:
        return '6,000 - 10,000'
    else:
        return '>10,000'

In [15]:
def dur_eval(x):
    if x <= 6:
        return '<= 6'
    elif x <= 12:
        return '6 - 12'
    elif x <= 24:
        return '12 - 24'
    elif x <= 36:
        return '24 - 36'
    elif x <= 50:
        return '36 - 50'
    else:
        return '>50'

In [16]:
#leyenda, 0 es default y 1 es no default
credit_woe['default'] = credit_woe['Risk'].map({'bad':0,'good':1})

In [17]:
credit_woe['conteo'] = 'a'

In [18]:
credit_woe['Credit amount'] = credit_woe['Credit amount'].apply(amt_eval)
credit_woe['Duration'] = credit_woe['Duration'].apply(dur_eval)
credit_woe['Age'] = credit_woe['Age'].apply(age_eval)

In [19]:
#función para definir el poder predictivo de las variables
def poder_predictivo(x):
    if x < 0.02:
        return 'no tiene'
    elif x <= 0.1:
        return 'debil'
    elif x <= 0.3:
        return 'medio'
    elif x <= 0.5:
        return 'fuerte'
    elif x > 0.5:
        return 'Sospechoso'
    else:
        return 'N/A'

In [20]:
#Función para obtener el Weight of Evidence de las variables 
def WoE(df_,variable,variable_default,variable_contar):
    df_ = df_.groupby([variable]).agg({variable_default:'mean',variable_contar:'count'}).reset_index()
    df_.columns = [variable,'prop_no_default','cantidad']
    df_['prop_cantidad'] = df_['cantidad']/df_['cantidad'].sum()
    df_['no_default'] = df_['prop_no_default']*df_['cantidad']
    df_['default'] = (1 - df_['prop_no_default'])*df_['cantidad']
    df_['prop_default'] = df_['default']/df_['default'].sum()
    df_['prop_no_default'] = df_['no_default']/df_['no_default'].sum()
    df_ = df_[(df_['default'] !=0) & (df_['no_default'] !=0)]
    df_['WoE'] = np.log(df_['prop_no_default']/df_['prop_default'])
    df_['IV'] = (df_['prop_no_default']-df_['prop_default'])*df_['WoE']
    df_['IV'] = df_['IV'].sum()
    df_['poder_predictivo'] = df_['IV'].apply(poder_predictivo)
    df_['variable'] = variable
    df_.columns = ['rango','prop_no_default',
                   'cantidad','prop_cantidad','no_default','default','prop_default',
                  'WoE','IV','poder_predictivo','variable']
    return df_

In [21]:
WoE(credit_woe,'Age','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,27 - 33,0.215714,225,0.225,151.0,74.0,0.246667,-0.134083,0.077301,debil,Age
1,33 - 42,0.271429,249,0.249,190.0,59.0,0.196667,0.322189,0.077301,debil,Age
2,42 - 52,0.154286,139,0.139,108.0,31.0,0.103333,0.400846,0.077301,debil,Age
3,< 27,0.262857,291,0.291,184.0,107.0,0.356667,-0.305191,0.077301,debil,Age
4,>52,0.095714,96,0.096,67.0,29.0,0.096667,-0.009901,0.077301,debil,Age


In [22]:
WoE(credit_woe,'Sex','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,female,0.287143,310,0.31,201.0,109.0,0.363333,-0.235341,0.026543,debil,Sex
1,male,0.712857,690,0.69,499.0,191.0,0.636667,0.113035,0.026543,debil,Sex


In [23]:
WoE(credit_woe,'Housing','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,free,0.091429,108,0.108,64.0,44.0,0.146667,-0.472604,0.083293,debil,Housing
1,own,0.752857,713,0.713,527.0,186.0,0.620000,0.194156,0.083293,debil,Housing
2,rent,0.155714,179,0.179,109.0,70.0,0.233333,-0.404445,0.083293,debil,Housing


In [24]:
WoE(credit_woe,'Saving accounts','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,little,0.767143,786,0.786,537.0,249.0,0.830000,-0.078753,0.077516,debil,Saving accounts
1,moderate,0.098571,103,0.103,69.0,34.0,0.113333,-0.139552,0.077516,debil,Saving accounts
2,quite rich,0.074286,63,0.063,52.0,11.0,0.036667,0.706051,0.077516,debil,Saving accounts
3,rich,0.060000,48,0.048,42.0,6.0,0.020000,1.098612,0.077516,debil,Saving accounts


In [25]:
WoE(credit_woe,'Checking account','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,little,0.198571,320,0.320,139.0,181.0,0.603333,-1.111321,0.740419,Sospechoso,Checking account
1,moderate,0.731429,617,0.617,512.0,105.0,0.350000,0.737066,0.740419,Sospechoso,Checking account
2,rich,0.070000,63,0.063,49.0,14.0,0.046667,0.405465,0.740419,Sospechoso,Checking account


In [26]:
WoE(credit_woe,'Credit amount','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,"1,000 - 2,000",0.331429,316,0.316,232.0,84.0,0.280000,0.168623,0.159674,medio,Credit amount
1,"2,000 - 3,000",0.204286,188,0.188,143.0,45.0,0.150000,0.308884,0.159674,medio,Credit amount
2,"3,000 - 4,000",0.150000,134,0.134,105.0,29.0,0.096667,0.439367,0.159674,medio,Credit amount
3,"4,000 - 6,000",0.085714,97,0.097,60.0,37.0,0.123333,-0.363871,0.159674,medio,Credit amount
4,"6,000 - 10,000",0.092857,109,0.109,65.0,44.0,0.146667,-0.457100,0.159674,medio,Credit amount
5,"< 1,000",0.112857,116,0.116,79.0,37.0,0.123333,-0.088768,0.159674,medio,Credit amount
6,">10,000",0.022857,40,0.040,16.0,24.0,0.080000,-1.252763,0.159674,medio,Credit amount


In [27]:
WoE(credit_woe,'Duration','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,12 - 24,0.412857,411,0.411,289.0,122.0,0.406667,0.015108,0.227127,medio,Duration
1,24 - 36,0.122857,143,0.143,86.0,57.0,0.190000,-0.436002,0.227127,medio,Duration
2,36 - 50,0.048571,71,0.071,34.0,37.0,0.123333,-0.931855,0.227127,medio,Duration
3,6 - 12,0.300000,277,0.277,210.0,67.0,0.223333,0.295117,0.227127,medio,Duration
4,<= 6,0.104286,82,0.082,73.0,9.0,0.030000,1.245937,0.227127,medio,Duration
5,>50,0.011429,16,0.016,8.0,8.0,0.026667,-0.847298,0.227127,medio,Duration


In [28]:
WoE(credit_woe,'Purpose','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,business,0.090000,97,0.097,63.0,34.0,0.113333,-0.230524,0.067309,debil,Purpose
1,car,0.330000,337,0.337,231.0,106.0,0.353333,-0.068319,0.067309,debil,Purpose
2,domestic appliances,0.011429,12,0.012,8.0,4.0,0.013333,-0.154151,0.067309,debil,Purpose
3,education,0.051429,59,0.059,36.0,23.0,0.076667,-0.399273,0.067309,debil,Purpose
4,furniture/equipment,0.175714,181,0.181,123.0,58.0,0.193333,-0.095557,0.067309,debil,Purpose
5,radio/TV,0.311429,280,0.280,218.0,62.0,0.206667,0.410063,0.067309,debil,Purpose
6,repairs,0.020000,22,0.022,14.0,8.0,0.026667,-0.287682,0.067309,debil,Purpose
7,vacation/others,0.010000,12,0.012,7.0,5.0,0.016667,-0.510826,0.067309,debil,Purpose


In [29]:
WoE(credit_woe,'Job_des','default','conteo')

,rango,prop_no_default,cantidad,prop_cantidad,no_default,default,prop_default,WoE,IV,poder_predictivo,variable
0,highly_skilled,0.138571,148,0.148,97.0,51.0,0.170000,-0.204413,0.008763,no tiene,Job_des
1,skilled,0.634286,630,0.630,444.0,186.0,0.620000,0.022780,0.008763,no tiene,Job_des
2,unskilled_no_resident,0.021429,22,0.022,15.0,7.0,0.023333,-0.085158,0.008763,no tiene,Job_des
3,unskilled_resident,0.205714,200,0.200,144.0,56.0,0.186667,0.097164,0.008763,no tiene,Job_des


# Fin